In [1]:
import numpy as np
import matplotlib.pyplot as plt

import time

#need to pip install pylorentz. it helps handle lorentz tranforms and 4vector creation.
#https://pylorentz.readthedocs.io/en/latest/_modules/pylorentz.html (source code)
from pylorentz import Momentum4 as mom4 #momentum4 is a 4vector function
import csv

import pandas as pd
import os
import glob

# Constants

In [2]:
hbar = 1 #natural units*);
c = 1  #natural units*);

ep = 10**(-6)

alph=1/137.04 #Fine Structure Constant

P = 120 #energy of incoming proton/basically the same as its incoming momentum (mp is very small)
POT = 1.44*10**(18)

mp=0.938 #GeV Proton Mass
m_mu = 0.1057 #GeV muon mass
#m_mu = 0.000511 #GeV muon mass
m_e = 0.000511 #GeV electron mass

# Form Factor and Constants

In [3]:
#form factor constants
mrh = 0.77
mome = 0.77
mrhop = 1.25
mrhopp = 1.45
momep = 1.25
momepp = 1.45
gammarho = 0.15
gammaome = 0.0085
gammarhop = 0.3
gammaomep = 0.3
gammarhopp = 0.5
gammaomepp = 0.5

#niverville /stefania's notebook
f1ra = 3.02/5.03
f1rb = 0.22320420111672623
f1rc = -0.33973820442685326
f1wa = 17.2/17.1
f1wb = -0.8816565944110686
f1wc = 0.3699021157531611

#cross section constants/ also from niverville
rD = 0.8/0.197
Hpp = 0.2704
Mpp = 2.2127
eta1pp = 0.451
eta2pp = 0.549
R1pp = 12.98
R2pp = 7.38
Ppp = 34.49

# Integrand calc -- > from ref (1609.01770)

In [4]:
###form factor functions

def F1r(mA):
    result = ((f1ra * mrh**2) / (mrh**2 - (mA + 1j*mrh*gammarho)) 
    
    + (f1rb * mrhop**2) / (mrhop**2 - (mA + 1j*mrhop*gammarhop))
    
    + (f1rc * mrhopp**2) / (mrhopp**2 - (mA + 1j*mrhopp*gammarhopp)))
    return result

def F1w(mA):
    result = ((f1wa * mome**2) / (mome**2 - (mA + 1j*mome*gammaome))
    
    + (f1wb * momep**2) / (momep**2 - (mA +1j*momep*gammaomep))
    
    + (f1wc*momepp**2) / (momepp**2 - (mA + 1j*momepp*gammaomepp)))
    
    return result
    
def F1proton(mA):
    
    result = np.abs((F1r(mA) +F1w(mA))**2)
    
    return result


### splitting function

def wpp(mA,z,kp2):
    
    H = kp2+(1-z)*mA**2 + (z*mp)**2
    
    prefactor = 1/(4*2*np.pi*H)
    
    result = (prefactor * ((1+(1-z)**2)/z - 2*z*(1-z) 
    * ((2*mp**2 + mA**2)/H - 2*((z*mp**2)/H)**2)
    
    + 2*z*(1-z)*(z+(1-z)**2)*(mp*mA/H)**2 
    
    + 2*z*((1-z)*mA**2/H)**2))
    
    return result

#cross section as function of sqr of center of mass energy

def sigmapp(s):
    
    sppM = (2*mp + Mpp)**2
    
    result = (Hpp*(np.log(s/sppM))**2 + Ppp 
              
    + R1pp*(s/sppM)**(-eta1pp)

    - R2pp*(s/sppM)**(-eta2pp))
    
    return result

#differential cross section. eqn (6) from (1609.01770)
             
def diffCrossIntegrand(mA, z, kp2):
    
    form = F1proton(mA**2)
    
    sp = 2*mp*(P - np.sqrt(mA**2 +kp2 +z**2*(P**2-mp**2)))
    
    s = 2*mp*P
    
    cross_ratio = sigmapp(sp)/sigmapp(s)
    
    integrand = ep**2 * alph * form * cross_ratio * wpp(mA,z,kp2)
    
    return integrand

# Generating dark photon -> di-muon events

We first generate the cross section by Monte Carlo integration with uniform event sampling. Then we determine a weighting for each sampled event (defined by its kp2 and z values) based on its contribution to the total cross section calculation. Each individual cross section value that contributes to the total cross section is divided by the total cross section to normalize weightings. We sample from this new, non-uniform distribution.

The 'events' function loops over each dark photon mass value and generates events at that mass value. It will save a csv's for each mass value. We will concatenate all the events at the very end.

In [5]:
#this function generates the dark photon to di-muon events
    
### dark photon mass vals of interest  

#masses from 0.25 to 0.55 in steps of 0.02
masses_1 = np.arange(0.25, 1.25, 0.02)
#masses from 0.55 to 3.05 in steps of 0.1
masses_2 = np.arange(0.65, 3.06, 0.1)
masses_e=[0.0011,0.0013,0.0016,0.0020,0.0025,0.0030,0.0035,0.0040,0.0050,0.007,0.010,0.02,0.04,0.06,0.08,0.10,0.12,0.14,0.16,0.18,0.20,0.22, 0.25, 0.27, 0.29, 0.31, 0.33, 0.35, 0.37, 0.39, 0.41, 0.43, 0.45, 0.47, 0.49, 0.51, 0.53, 0.55,0.57,0.59,0.61,0.63,0.65,0.67,0.69,0.71,0.73,0.75,0.77,0.79,0.81,0.83,0.85,0.87,0.89,0.91,0.93,0.95,0.97,0.99,1.01,1.03,1.05,1.07,1.09,1.11,1.13,1.15,1.17,1.19,1.21,1.23,1.25,1.27,1.29,1.31,1.33,1.35,1.37,1.39,1.41,1.43,1.45,1.47,1.49]
masses_mu=[0.22, 0.25, 0.27, 0.29, 0.31, 0.33, 0.35, 0.37, 0.39, 0.41, 0.43, 0.45, 0.47, 0.49, 0.51, 0.53, 0.55,0.57,0.59,0.61,0.63,0.65,0.67,0.69,0.71,0.73,0.75,0.77,0.79,0.81,0.83,0.85,0.87,0.89,0.91,0.93,0.95,0.97,0.99,1.01,1.03,1.05,1.07,1.09,1.11,1.13,1.15,1.17,1.19,1.21,1.23,1.25,1.27,1.29,1.31,1.33,1.35,1.37,1.39,1.41,1.43,1.45,1.47,1.49]
masses=np.array(masses_mu)
# Concatenate the masses
#x = np.concatenate([masses_1, masses_2])
x = np.concatenate([masses])


def events(f, N, n): 
    
###cross section vals to append 
    cross=[]
   
    #timer
    start = time.time()
    #iteration
    iter_ = 0
    
    
### looping over dark photon mass vals
    for j in range(len(x)):
        
        iter_ +=1
        print(f'iteration: {iter_}' )
        
        #to append with events at each mass value
        events = []
    
        start1 = time.time()
        
        
###     making N length arrays of arguments of integrand at each mass val

        #N length array of the mass value
        m = np.full(N,x[j])
        #kp2 vals; square of transverse momentum
        kp2 = np.random.uniform(0,1,N)
        #constant z bounds case; ratio of z-momenta of dark photon to initial beam particle momentum
        z = np.random.uniform(0.1,0.9,N) 
        

###     cross section calulation. 

        #making array of integrand evaluations at each set of variables
        #z range is 0.9, kp2 range is 1
        func = 0.9*1*4*POT*f(m,z,kp2)/N  

        
        #monte carlo method of summing each integrand output then dividing by the total number of samples, N. 
        #N is already divided in func above
        sum1 = np.sum(func)
       
        
        #appending w/cross sections
        cross.append(sum1)
        
        
        #dividing integrand evaluated at each set of variables by the total sum to find their weightings
        weights = func/sum1
        
        #making sure weights sum to 1
        print(f'weights sum to {np.sum(weights)}')
        
        

###     new event sample based on the weighting of each sampled event on the total cross section calculation           
        
        #COUPLED kinematics w/ weights
        #sampling indexes based on weights. Taking n samples
        ind = np.arange(0,N)
        ind_samples = np.random.choice(ind, size=n, p=weights)
        
        #taking the corresponding indexed variables
        kp2_samples = kp2[ind_samples]
    
        z_samples = z[ind_samples]
        
        #masses for new size n samples
        m_samples = m[:n]     
        
        
           
###     generating dark photon events
        
        #first starting with dark photon events in lab frame
        #random sampling azimuthal angles from 0 to 2pi
        phi = np.random.uniform(0,2 * np.pi, n)
        
        #x, y, z components of kp vals and energy (4-momentum)
        kp_x = np.sqrt(kp2_samples) * np.cos(phi)
        kp_y = np.sqrt(kp2_samples) * np.sin(phi)        
        kp_z = z_samples * 120
        E = np.sqrt(kp_x**2 + kp_y**2 + kp_z**2 + m_samples**2)
        
        # making array of momentum 4 vectors for A' (dark photon)
        A_m4 = []
        for i in range(n):
            momentum4 = mom4(E[i],kp_x[i],kp_y[i],kp_z[i])  #mom4 is a 4vector function from pylorentz package
            A_m4.append(momentum4)


###     generating di-muon events in rest frame of dark photon

        #momentum magnitude. conservation of energy and momentum in rest frame of dark photon
        P_mag = np.sqrt((1/4)*x[j]**2 - (1/1)*m_mu**2)
        
        #now we enter rest frame of dark photon and generate muon decays
        #random sampling azimuthal angles from 0 to 2pi for muon decays
        phi_mu = np.random.uniform(0,2 * np.pi,n)
        #sampling polar angles such that cos(theta) ranges from [-1,1] uniformly. 
        #Uniform sampling the polar angle leads to preference on the poles of a sphere
        polar =  np.random.uniform(-1,1,n)
        theta_mu = np.arccos(polar)
        
        #muon momentum
        p_mu1_x = np.sin(theta_mu)*np.cos(phi_mu)*P_mag
        p_mu1_y = np.sin(theta_mu)*np.sin(phi_mu)*P_mag
        p_mu1_z = np.cos(theta_mu)*P_mag
        
        #anti-muon momentum
        p_mu2_x = -p_mu1_x
        p_mu2_y = -p_mu1_y
        p_mu2_z = -p_mu1_z
        
        #muon energy (same for muon and anti-muon)
        E_mu = np.sqrt(p_mu1_x**2 + p_mu1_y**2 + p_mu1_z**2 +m_mu**2)
        
###     boosting muons into lab frame
    
        #making lists of muon and anti-muon 4-momentums
        mu_m4 = [] #muon
        anti_mu_m4 = [] #anti-muon
        for i in range(n):
            mu1_m4 = mom4(E_mu[i],p_mu1_x[i],p_mu1_y[i],p_mu1_z[i]) 
            mu_m4.append(mu1_m4)
            
            mu2_m4 = mom4(E_mu[i],p_mu2_x[i],p_mu2_y[i],p_mu2_z[i]) 
            anti_mu_m4.append(mu2_m4)
            
        #arrays to boost muon and anti-muons to corresponding dark photon lab frame   
        mu_m4_boost = []
        anti_mu_m4_boost = []
        
        #making arrays separating the bigger and smaller tranverse momentums 
        pt_high = []
        pt_low = []
        
        #arrays for separating into high and low z momenta
        z_high = []
        z_low = []
        
        #all z momenta
        z = []
        
        #separating the muons into high and low momenta
        for i in range(n):
            mu1_boost = mu_m4[i].boost_particle(A_m4[i]) #boost muon into lab frame
            mu_m4_boost.append(mu1_boost)
            
            mu2_boost = anti_mu_m4[i].boost_particle(A_m4[i]) #boost anti-muon into lab frame
            anti_mu_m4_boost.append(mu2_boost)
            
            #muon and antimuon transverse momenta. m1 is muon, m2 is antimuon
            mu1_pt = np.sqrt(mu1_boost.components[1]**2 + mu1_boost.components[2]**2)
            mu2_pt = np.sqrt(mu2_boost.components[1]**2 + mu2_boost.components[2]**2)
            
            #the z momenta
            mu1_pz = mu1_boost.components[3]
            mu2_pz = mu2_boost.components[3]            
            

            #separating momentas
            pt_high.append(max(mu1_pt,mu2_pt))
            pt_low.append(min(mu1_pt,mu2_pt))
            
            z_high.append(max(mu1_pz,mu2_pz))
            z_low.append(min(mu1_pz,mu2_pz))
    
            #adding to array of all z momenta
            z.append(mu1_pz)
            z.append(mu2_pz)
            
            
            
        #checking invariant mass of muon  
        print(f'muon mass: {np.sqrt(mu_m4_boost[0].components[0]**2 - (mu_m4_boost[0].components[1]**2 + mu_m4_boost[0].components[2]**2 + mu_m4_boost[0].components[3]**2))} GeV')
        print(f'anti-muon mass: {np.sqrt(mu_m4_boost[1].components[0]**2 - (mu_m4_boost[1].components[1]**2 + mu_m4_boost[1].components[2]**2 + mu_m4_boost[1].components[3]**2))} GeV')
        
        
        #this is for organizing data into table format
        for i in range(n):
            A_id = 666
            mu1_id = 13
            mu2_id = -13
            #mu1_id = 11
            #mu2_id = -11
                        
            A_stat = 2
            mu_stat = -1
            
            A_parents = 0
            mu_parents = 1
            
            color = 0
            
            A_spin = 1
            mu_spin = 1/2
            
            #energy
            E_A = A_m4[i].components[0]
            E_mu1 = mu_m4_boost[i].components[0]
            E_mu2 = anti_mu_m4_boost[i].components[0]
            
            #x momenta
            px_A = A_m4[i].components[1]
            px_mu1 = mu_m4_boost[i].components[1]
            px_mu2 = anti_mu_m4_boost[i].components[1]
            
            #y momenta
            py_A = A_m4[i].components[2]
            py_mu1 = mu_m4_boost[i].components[2]
            py_mu2 = anti_mu_m4_boost[i].components[2]
            
            #z momenta
            pz_A = A_m4[i].components[3]
            pz_mu1 = mu_m4_boost[i].components[3]
            pz_mu2 = anti_mu_m4_boost[i].components[3]
            
            #dark photon mass and muon mass
            m_A = x[j]
            m_mu1 = np.sqrt(mu_m4_boost[i].components[0]**2 - (mu_m4_boost[i].components[1]**2 + mu_m4_boost[i].components[2]**2 + mu_m4_boost[i].components[3]**2))
            m_mu2 = np.sqrt(anti_mu_m4_boost[i].components[0]**2 - (anti_mu_m4_boost[i].components[1]**2 + anti_mu_m4_boost[i].components[2]**2 + anti_mu_m4_boost[i].components[3]**2))
            
            events.append([A_id,A_stat,A_parents,color,E_A,px_A,py_A,pz_A,m_A,A_spin])
            events.append([mu1_id,mu_stat,mu_parents,color,E_mu1,px_mu1,py_mu1,pz_mu1,m_mu1,mu_spin])
            events.append([mu2_id,mu_stat,mu_parents,color,E_mu2,px_mu2,py_mu2,pz_mu2,m_mu2,mu_spin])
        

            
       ###plotting histograms for muon events
        
        
#         fig, ((ax1, ax2), (ax3,ax4)) = plt.subplots(2,2 , figsize=(8, 6))  
       
#         #high transverse momenta
#         xcounts, xbins, x_ = ax1.hist(pt_high, bins=30, density=False)
#         ax1.set_title(f'pt_high, muon, m = {x[j]} GeV')
#         ax1.set_ylabel('Counts')
#         ax1.set_xlabel('pt_high')
#         ax1.set_aspect('auto') 
        
#         # low transverse momenta
#         ycounts, ybins, y_ = ax2.hist(pt_low, bins=30, density=False)
#         ax2.set_title(f'pt_low, muon, m = {x[j]} GeV')
#         ax2.set_ylabel('Counts')
#         ax2.set_xlabel('pt_low')
#         ax2.set_aspect('auto') 
        
#         # high z momenta
#         z_h_counts, z_h_bins, z_h_ = ax3.hist(z_high, bins=30, density=False)
#         ax3.set_title(f'p_z_high, muon, m = {x[j]} GeV')
#         ax3.set_ylabel('Counts')
#         ax3.set_xlabel('p_z_high')
#         ax3.set_aspect('auto')
        
#         # low z momenta
#         z_l_counts, z_l_bins, z_l_ = ax4.hist(z_low, bins=30, density=False)
#         ax4.set_title(f'p_z_low, muon, m = {x[j]} GeV')
#         ax4.set_ylabel('Counts')
#         ax4.set_xlabel('p_z_low')
#         ax4.set_aspect('auto') 
        
#         plt.tight_layout()
#         plt.show()
        
    
    
        #this is to save event files. they are saved separately for every mass value in case of memory issues
        with open(f'di_muon_events{n}_{j}.csv', 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerows(events)
    
    

        end1 = time.time()
        print(f'loop time: {end1-start1} sec')
        print()
        
    
    #stop timer
    end = time.time()
    time_elapsed = end-start
    
    print(f'total time: {time_elapsed} sec')
    
    
    
    
    #plotting cross section
#     plt.yscale("log")
#     plt.xscale("log")
#     plt.plot(x,cross)
#     plt.show()    

    
    
### events are generated for each mass value then saved to save memory. So, we concatenate them all into one dataset here
    
    # empty list to hold the data frames
    dfs = []

    header = ['PDG ID', 'status','parents','color', 'E[GeV]', 'px[GeV]', 'py[GeV]', 'pz[GeV]', 'mass', 'spin']
 

    #loop over the csv file for all of the masses
    for i in range(len(x)):  
        m = x[i]
        filename = f'di_muon_events{n}_{i}.csv'
        if os.path.isfile(filename):  # Check if the file exists
            # Read the CSV file without headers
            df = pd.read_csv(filename, header=None)
            df.to_csv(f'dimuon_events{n}_{round(m,4)}.csv', index=False, header=header)
            #df.to_csv(f'dielectron_events{n}_{round(m,4)}.csv', index=False, header=header)

            dfs.append(df)


    # Concatenate all data frames in the list vertically
    concatenated_df = pd.concat(dfs, axis=0)
    
    # Save the concatenated data frame to a new CSV file with the header
    concatenated_df.to_csv(f'di_muon_events{n}.csv', index=False, header=header)
    
    
### now delete all of the individual csv files that were made intermediately

    # Define the filename pattern for the files to be deleted
    pattern = f"di_muon_events{n}_*.csv"

    # Get a list of all files matching the pattern in the current directory
    files_to_delete = glob.glob(pattern)

    # Delete all files matching the pattern except the concatenated one
    #for file_to_delete in files_to_delete:
    #    if file_to_delete != f"di_electron_events{n}.csv":
    #        os.remove(file_to_delete)


In [6]:
nevents = 10000
events(diffCrossIntegrand,1000000,nevents)

#the first argument is the function to integrate over

#second argument is the initial sample size for the uniform monte carlo integration

#third argument is the number of events desired from the weighted distribution of kinematic events at each mass value

iteration: 1
weights sum to 0.9999999999999993
muon mass: 0.10569999999986493 GeV
anti-muon mass: 0.10570000000003299 GeV
loop time: 3.2524542808532715 sec

iteration: 2
weights sum to 1.0000000000000002
muon mass: 0.10570000000231855 GeV
anti-muon mass: 0.10570000000016744 GeV
loop time: 2.8865911960601807 sec

iteration: 3
weights sum to 1.0000000000000002
muon mass: 0.10569999999936076 GeV
anti-muon mass: 0.10570000000016744 GeV
loop time: 3.2487757205963135 sec

iteration: 4
weights sum to 0.9999999999999994
muon mass: 0.10569999999936076 GeV
anti-muon mass: 0.10570000000016744 GeV
loop time: 3.2819406986236572 sec

iteration: 5
weights sum to 1.0000000000000002
muon mass: 0.10569999999909187 GeV
anti-muon mass: 0.10569999999999938 GeV
loop time: 3.198207378387451 sec

iteration: 6
weights sum to 1.0000000000000002
muon mass: 0.10569999999994896 GeV
anti-muon mass: 0.10569999999996577 GeV
loop time: 3.7434277534484863 sec

iteration: 7
weights sum to 0.9999999999999999
muon mass: 0

muon mass: 0.10570000000003299 GeV
anti-muon mass: 0.10569999999989854 GeV
loop time: 3.0970895290374756 sec

iteration: 55
weights sum to 1.000000000000001
muon mass: 0.10569999999909187 GeV
anti-muon mass: 0.105700000001243 GeV
loop time: 2.7838432788848877 sec

iteration: 56
weights sum to 1.0000000000000002
muon mass: 0.1057000000000666 GeV
anti-muon mass: 0.10570000000004139 GeV
loop time: 2.890289545059204 sec

iteration: 57
weights sum to 0.9999999999999998
muon mass: 0.10569999999909187 GeV
anti-muon mass: 0.10570000000769636 GeV
loop time: 2.9563159942626953 sec

iteration: 58
weights sum to 1.0000000000000002
muon mass: 0.10570000000043632 GeV
anti-muon mass: 0.10570000000016744 GeV
loop time: 3.1093807220458984 sec

iteration: 59
weights sum to 1.0
muon mass: 0.10570000000016744 GeV
anti-muon mass: 0.10569999999801631 GeV
loop time: 3.7295408248901367 sec

iteration: 60
weights sum to 1.0000000000000004
muon mass: 0.10569999999936076 GeV
anti-muon mass: 0.10569999999994896 G

In [7]:
import csv

def convert_csv_to_lhe(csv_filename, lhe_filename):
    with open(csv_filename, mode='r') as csv_file, open(lhe_filename, mode='w') as lhe_file:
        csv_reader = csv.reader(csv_file)
        # Read the events
        event_lines = []
        for row in csv_reader:
            if row[0] == "PDG ID": 
                formatted_row = ' '.join(f'{item:<18}' if i > 3 else f'{item:<10}' for i, item in enumerate(row)) 
                lhe_file.write(formatted_row + '\n')
                continue
            if row[0] == '666':
                if event_lines:
                    lhe_file.write('<event>\n')
                    for event_line in event_lines:
                        lhe_file.write(event_line + '\n')
                    lhe_file.write('</event>\n')
                    event_lines = []
                formatted_row = ' '.join(f'{float(item):<18}' if i > 3 else f'{int(item):<10}' for i, item in enumerate(row))
                #formatted_row=row.replace(",", " ")
                event_lines.append(formatted_row)
            else:
                formatted_row = ' '.join(f'{float(item):<18}' if i > 3 else f'{int(item):<10}' for i, item in enumerate(row))
                #formatted_row=row.replace(",", " ")
                event_lines.append(formatted_row)

        # Write the last event
        if event_lines:
            lhe_file.write('<event>\n')
            for event_line in event_lines:
                lhe_file.write(event_line + '\n')
            lhe_file.write('</event>\n')


In [8]:
for i in x:
    convert_csv_to_lhe("dimuon_events"+str(nevents)+"_"+str(round(i,4))+".csv","SpinQuestAprimeToMuonsLHE_Brem_mAp_"+str(round(i,4))+"_GeV.txt")    #convert_csv_to_lhe("dimuon_events"+str(nevents)+"_"+str(round(i,3))+".csv","SpinQuestAprimeToMuonsLHE_Brem_mAp_"+str(round(i,3))+"_GeV.txt")
    #convert_csv_to_lhe("dielectron_events"+str(nevents)+"_"+str(round(i,4))+".csv","SpinQuestAprimeToElectronsLHE_Brem_mAp_"+str(round(i,4))+"_GeV.txt")    #convert_csv_to_lhe("dimuon_events"+str(nevents)+"_"+str(round(i,3))+".csv","SpinQuestAprimeToMuonsLHE_Brem_mAp_"+str(round(i,3))+"_GeV.txt")